In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from connect_to_mysql import Database
import networkx as nx
import matplotlib.pyplot as plt
import collections 
pd.set_option("display.max_columns", None)

In [ ]:
#repo = git.Repo(".", search_parent_directories=True).git.rev_parse("--show-toplevel")
#df = pd.read_csv(f"{repo}/data/cleaning/test_1651834681.csv")
df = pd.read_csv("/Users/ericchittka/Documents/GitHub/vehicle-stream-team-project/data/mtd_combined.csv")
df = df[(df["state"] == "completed")]

In [ ]:
Database.query_dataframe('rides', 'Show tables')

In [ ]:
#df_edges = Database.query_dataframe('stopconnection', "Select * From stopconnection")
df_edges = pd.read_excel("/Users/ericchittka/Documents/Daten/MoDstops+Preismodell.xlsx")

In [ ]:
df_edges.dtypes

### Number of Drives 

In [ ]:
df.columns

In [ ]:
pd.to_datetime(df["scheduled_to"])[0].month

In [ ]:
frequencies = pd.DataFrame(df[(pd.to_datetime(df["scheduled_to"]).dt.month == 2) & (pd.to_datetime(df["scheduled_to"]).dt.year == 2022) ].groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()

frequencies

In [ ]:
frequencies["pickup_address"] = frequencies["pickup_address"].astype(str)
frequencies["dropoff_address"] = frequencies["dropoff_address"].astype(str)

In [ ]:
#df_edges = pd.read_excel(
#        f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name="Liste 2022"
#)

In [ ]:
df_edges.columns

In [ ]:
df_edges = df_edges[["id_start", "id_end"]]

In [ ]:
df_edges["id_start"] = df_edges["id_start"].astype(str)
df_edges["id_end"] = df_edges["id_end"].astype(str)

In [ ]:
edges = pd.merge(df_edges, frequencies,  how='left', left_on=['id_start','id_end'], right_on = ['pickup_address','dropoff_address'])[["id_start", "id_end", "number_of_drives"]]
edges = edges[edges["number_of_drives"].isna() == False]
edges["number_of_drives"] = 28  / edges["number_of_drives"] 

In [ ]:
edges

In [ ]:
edges["number_of_drives"].fillna(0, inplace=True)

In [ ]:
#G = nx.from_pandas_edgelist(
#    edges,
#    source = "id_start",
#    target = "id_end",
#    edge_attr= "number_of_drives"
#)

In [ ]:
#plt.figure(3,figsize=(30,30)) 
#nx.draw(G)
#plt.show()

In [ ]:
#print(nx.shortest_path(G, source="10002", target="4007", weight = "number_of_drives"))
#print(nx.shortest_path_length(G, source="10002", target="4007", weight="number_of_drives", method='dijkstra'))

### Average Ride Time per drive  

In [ ]:
#df
#df.dtypes['ride_time']

In [ ]:
average = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean())
average.rename(columns = {"ride_time":"avg_ride_time"}, inplace = True)
average

In [ ]:
average["pickup_address"] = average["pickup_address"].astype(str)
average["dropoff_address"] = average["dropoff_address"].astype(str)

In [ ]:
edges2 = pd.merge(df_edges, average,  how='left', left_on=['id_start','id_end'], right_on = ['pickup_address','dropoff_address'])[["id_start", "id_end", "avg_ride_time"]]
edges2 = edges2[edges2["avg_ride_time"].isna() == False]
edges2

In [ ]:
#G1 = nx.from_pandas_edgelist(
#    edges2,
#    source = "id_start",
#    target = "id_end",
#    edge_attr= "avg_ride_time"
#)

In [ ]:
#plt.figure(3,figsize=(30,30)) 
#nx.draw(G1)
#plt.show()

In [ ]:
#print(nx.shortest_path(G1, source="1001", target="9015", weight = "avg_ride_time"))
#print(nx.shortest_path_length(G1, source="1001", target="9015", weight="avg_ride_time", method='dijkstra'))

### Original Function

In [ ]:
def get_shortest_ride(startpoint, endpoint, start_date, end_date):
    start_date = dt.strptime(start_date, "%m/%d/%Y")
    end_date = dt.strptime(end_date, "%m/%d/%Y")
    days = (end_date - start_date).days + 1
    df = Database.query_dataframe('rides', "Select * From rides where state = 'completed'")
    df = df[(df["scheduled_to"] > start_date) & (df["scheduled_to"] < end_date)]
    
    drives = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()
    
    drives["waiting_time"] = days / drives["number_of_drives"]

    drives["avg_ride_time"] = df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean()["ride_time"] / 60 / 60 / 24
    
    drives["avg_time_to_destination"] = drives["waiting_time"] + drives["avg_ride_time"]

    G = nx.from_pandas_edgelist(
        drives,
        source = "pickup_address",
        target = "dropoff_address",
        edge_attr= "avg_time_to_destination"
    )

    path = nx.shortest_path(G, source=startpoint, target=endpoint, weight = "avg_time_to_destination")
    shortest_time = nx.shortest_path_length(G, source=startpoint, target=endpoint, weight="avg_time_to_destination", method='dijkstra') 

    return (path, shortest_time)

### Set start and end date 

In [2]:
df = pd.read_csv("/Users/ericchittka/Documents/GitHub/vehicle-stream-team-project/data/cleaning/test_1656252990.csv")
df = df[(df["state"] == "completed")]

In [3]:
#df = Database.query_dataframe('rides', "Select * From rides where state = 'completed'")
startdate = df.scheduled_to.min()
startdate = dt.strptime(startdate, '%Y-%m-%d %H:%M:%S')
startdate = startdate.strftime('%m/%d/%Y')
start_date = dt.strptime(startdate, "%m/%d/%Y")
enddate = df.scheduled_to.max()
enddate = dt.strptime(enddate, '%Y-%m-%d %H:%M:%S')
enddate = enddate.strftime('%m/%d/%Y')
end_date = dt.strptime(enddate, "%m/%d/%Y")

In [4]:
#df['pickup_address'] = df['pickup_address'].map(str)
#df['dropoff_address'] = df['dropoff_address'].map(str)

### Drive DF

In [5]:
days = (end_date - start_date).days + 1
drives = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()
drives["waiting_time"] = days / drives["number_of_drives"]
drives["avg_ride_time"] = df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean()["ride_time"] / 60 / 60 / 24
drives["avg_time_to_destination"] = drives["waiting_time"] + drives["avg_ride_time"]

In [6]:
drives

,pickup_address,dropoff_address,number_of_drives,waiting_time,avg_ride_time,avg_time_to_destination
0,1001,1004,5,54.800000,0.001738,54.801738
1,1001,1005,18,15.222222,0.001552,15.223774
2,1001,1008,3,91.333333,0.003538,91.336871
3,1001,1012,1,274.000000,0.007627,274.007627
4,1001,1013,1,274.000000,0.005856,274.005856
...,...,...,...,...,...,...
2271,18005,7001,4,68.500000,0.006704,68.506704
2272,18005,9001,1,274.000000,0.010185,274.010185
2273,18005,9005,6,45.666667,0.009236,45.675903
2274,18005,9006,1,274.000000,0.009155,274.009155


### Graph

In [7]:
G = nx.from_pandas_edgelist(
    drives,
    source = "pickup_address",
    target = "dropoff_address",
    edge_attr= "avg_time_to_destination"
)

In [8]:
G.nodes

NodeView((1001, 1004, 1005, 1008, 1012, 1013, 4018, 6004, 9005, 9015, 9019, 12011, 13003, 13005, 14006, 1002, 2007, 3018, 3023, 3024, 4007, 4008, 4025, 4027, 4041, 6002, 9006, 10004, 11012, 12010, 15013, 16009, 16027, 1003, 9018, 1007, 1014, 2008, 3001, 3004, 3010, 3021, 3025, 4006, 4037, 5001, 5002, 5003, 8001, 9001, 9002, 9009, 9011, 9021, 11002, 11003, 11005, 11008, 11017, 12003, 12004, 12005, 12007, 12008, 13001, 13002, 14003, 15007, 15009, 15011, 15012, 15015, 17007, 18005, 1006, 2002, 2005, 2006, 3006, 3009, 3011, 3013, 3014, 3015, 3019, 3020, 3026, 3027, 3028, 4001, 4013, 4024, 4026, 4029, 4031, 4033, 4034, 4035, 4039, 5006, 5009, 6003, 7001, 9004, 9017, 10001, 10003, 10005, 11007, 11009, 11011, 11014, 12001, 12009, 14002, 14011, 15002, 15004, 15005, 15006, 15014, 1009, 1010, 3002, 3016, 4030, 9003, 9010, 1011, 3007, 3017, 4012, 5007, 5008, 5010, 8002, 12012, 13004, 14005, 2001, 2004, 2009, 3012, 4004, 4016, 4020, 4023, 4038, 4040, 6001, 6005, 9008, 11010, 12002, 14001, 15001, 1

In [9]:
def calculate_graph(start_date, end_date):
  
    G = nx.from_pandas_edgelist(
        drives,
        source = "pickup_address",
        target = "dropoff_address",
        edge_attr= "avg_time_to_destination"
    )
    return G

In [10]:
def get_shortest_ride(startpoint, endpoint, G):

    if startpoint not in G:
        return 'Not in G'
    elif endpoint not in G: 
        return 'Not in G'
    else:
        path = nx.shortest_path(G, source=startpoint, target=endpoint, weight = "avg_time_to_destination")
        return path

In [11]:
# df_test = pd.DataFrame({'Start':['1001', '1002', '1931391'], 'End': ['10004', '1008', '32432432']})
# df_test["Start"] = df_test["Start"].astype(str).astype(int)
# df_test["End"] = df_test["End"].astype(str).astype(int)
# df_test['HotSpots'] = df_test.apply(lambda x: get_shortest_ride(x.Start, x.End, G), axis=1)
# df_test

In [12]:
df_edges = pd.read_excel("/Users/ericchittka/Documents/GitHub/vehicle-stream-team-project/data/other/MoDstops+Preismodell.xlsx", sheet_name= 'Liste 2022')

In [13]:
df_edges.rename(columns= {'Start #':'start_id', 'Ende #':'end_id'}, inplace = True)
df_edges['Spots'] = df_edges.apply(lambda x: get_shortest_ride(x.start_id, x.end_id, G), axis=1)

In [14]:
df_edges = df_edges[df_edges.Spots != 'Not in G']

In [15]:
hotSpots_list = list(df_edges['Spots'])

In [16]:
flat_hotSpots_list = [x for xs in hotSpots_list for x in xs]
flat_hotSpots_list

[1001,
 1008,
 1002,
 1001,
 1005,
 9018,
 1003,
 1001,
 1005,
 4025,
 1004,
 1001,
 1005,
 1001,
 1005,
 4025,
 1006,
 1001,
 1005,
 4025,
 1007,
 1001,
 1008,
 1001,
 1005,
 4025,
 1009,
 1001,
 1005,
 4025,
 1010,
 1001,
 1005,
 2006,
 1011,
 1001,
 1008,
 1002,
 1012,
 1001,
 1005,
 2006,
 1013,
 1001,
 1005,
 7001,
 1014,
 1001,
 1008,
 2001,
 1001,
 1008,
 14003,
 2002,
 1001,
 1005,
 4025,
 1007,
 2003,
 1001,
 1005,
 12007,
 2004,
 1001,
 1008,
 2005,
 1001,
 1005,
 2006,
 1001,
 1005,
 12007,
 2007,
 1001,
 1008,
 2008,
 1001,
 1008,
 2009,
 1001,
 1005,
 4025,
 3001,
 1001,
 1005,
 4025,
 1009,
 10005,
 3002,
 1001,
 1005,
 4013,
 3003,
 1001,
 1005,
 4025,
 1004,
 3004,
 1001,
 1005,
 9004,
 3005,
 1001,
 1008,
 11017,
 3010,
 3006,
 1001,
 1008,
 3007,
 1001,
 1005,
 3009,
 1001,
 1008,
 11017,
 3010,
 1001,
 1008,
 3011,
 1001,
 1005,
 4025,
 1009,
 10005,
 3012,
 1001,
 1005,
 4025,
 3013,
 1001,
 1008,
 11017,
 3014,
 1001,
 1005,
 7001,
 3015,
 1001,
 1005,
 4025,
 1006

In [17]:
counter=collections.Counter(flat_hotSpots_list)
print(counter.most_common(15))

[(1008, 20448), (4025, 20178), (1005, 10060), (1009, 6490), (1002, 4170), (1007, 4072), (12007, 3008), (7001, 2872), (6004, 2478), (14003, 2418), (3001, 2410), (11017, 2216), (1010, 1810), (10005, 1534), (1004, 1410)]


### Adding Drone flights 

In [18]:
drone_flights = df_edges.iloc[:,:6]
drone_flights.drop(['Start Name', 'Ende Name', 'Route [m]'], axis=1, inplace=True)
drone_flights.rename(columns = {'Luftlinie [m]':'Luftlinie'}, inplace = True)
drone_flights = drone_flights[drone_flights.Luftlinie <= 500]
drone_flights 

,start_id,end_id,Luftlinie
0,1001,1002,285
8,1001,1010,314
22,1001,3001,337
29,1001,3009,243
32,1001,3012,386
...,...,...,...
39382,15015,15001,434
39384,15015,15003,380
39385,15015,15004,421
39386,15015,15005,223


In [19]:
# Annahme: Drohne fliegt 30 km/h --> 12 Sekunden pro 100 Meter und dann in Tage umrechnen  
drone_flights['avg_time_to_destination'] = (drone_flights.Luftlinie * 0.12) /60 /60 /24

In [22]:
drone_flights

,start_id,end_id,Luftlinie,avg_time_to_destination
0,1001,1002,285,0.000396
8,1001,1010,314,0.000436
22,1001,3001,337,0.000468
29,1001,3009,243,0.000337
32,1001,3012,386,0.000536
...,...,...,...,...
39382,15015,15001,434,0.000603
39384,15015,15003,380,0.000528
39385,15015,15004,421,0.000585
39386,15015,15005,223,0.000310


In [23]:
drives

,pickup_address,dropoff_address,number_of_drives,waiting_time,avg_ride_time,avg_time_to_destination
0,1001,1004,5,54.800000,0.001738,54.801738
1,1001,1005,18,15.222222,0.001552,15.223774
2,1001,1008,3,91.333333,0.003538,91.336871
3,1001,1012,1,274.000000,0.007627,274.007627
4,1001,1013,1,274.000000,0.005856,274.005856
...,...,...,...,...,...,...
2271,18005,7001,4,68.500000,0.006704,68.506704
2272,18005,9001,1,274.000000,0.010185,274.010185
2273,18005,9005,6,45.666667,0.009236,45.675903
2274,18005,9006,1,274.000000,0.009155,274.009155


In [31]:
drives_w_flights = pd.merge(drives, drone_flights, left_on=["pickup_address", "dropoff_address"], right_on=["start_id", "end_id"], how="left")

In [32]:
drives_w_flights

,pickup_address,dropoff_address,number_of_drives,waiting_time,avg_ride_time,avg_time_to_destination_x,start_id,end_id,Luftlinie,avg_time_to_destination_y
0,1001,1004,5,54.800000,0.001738,54.801738,NaN,NaN,NaN,NaN
1,1001,1005,18,15.222222,0.001552,15.223774,NaN,NaN,NaN,NaN
2,1001,1008,3,91.333333,0.003538,91.336871,NaN,NaN,NaN,NaN
3,1001,1012,1,274.000000,0.007627,274.007627,NaN,NaN,NaN,NaN
4,1001,1013,1,274.000000,0.005856,274.005856,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2271,18005,7001,4,68.500000,0.006704,68.506704,NaN,NaN,NaN,NaN
2272,18005,9001,1,274.000000,0.010185,274.010185,NaN,NaN,NaN,NaN
2273,18005,9005,6,45.666667,0.009236,45.675903,NaN,NaN,NaN,NaN
2274,18005,9006,1,274.000000,0.009155,274.009155,NaN,NaN,NaN,NaN
